## Preamble

In [ ]:
# Required packages
import numpy as np
import cv2 
from scipy.signal import savgol_filter
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from math import *
import tifffile
import warnings
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from scipy.optimize import curve_fit
import string
from pylab import *
from PIL import Image
from osgeo import gdal as GD  
warnings.filterwarnings("ignore")

In [ ]:
# Plot formatting 
matplotlib.rcParams['mathtext.fontset']='cm'
matplotlib.rcParams['font.family']='STIXGeneral'
plt.rcParams['legend.fontsize']=15
plt.rcParams.update({'font.size':15}) 
plt.rcParams['axes.axisbelow']=True

# Thickness of Shear Band from Strain Data

### 1. Define function that calculates the weighted strain density perpendicular to shear band

In [ ]:
#### CALCULATE THE WEIGHTED (AREA) DENSITY 
conv = 7.91 # pixel resolution conversion
def density_calculation(step, weight, intercept, band_max, Y, size, title):
    '''
    step: integer -- space between horizontal bins
    weight: integer -- weight of strain density
    intercept: integer -- inclination of straight line that fits the shear band
    band_max: list -- distance to last band to which density is calculated
    size: pd.DataFrame -- strain dataframe 
    
    '''
    
    densities_p = []
    densities_n = []
    for b in range(0, band_max, step): 
        ### positive bands (ABOVE from shear band line)
        def condition(x):
            return x<= intercept+b+step and x>= intercept+b
        band_objects_y = [x for x in Y if condition(x)]
        band_major2 = []
        for i in range(len(band_objects_y)): 
            index_y = Y.index(band_objects_y[i])
            band_major2.append((size.iloc[index_y])**weight)
        AWden = sum(band_major2)
        densities_p.append(AWden)
        
        ### negative bands (BELOW from shear band line)
        def condition_n(a):
            return a>= intercept-b-step and a<= intercept-b
        band_objects_y_n = [a for a in Y if condition_n(a)]
        band_major2_n = []
        for o in range(len(band_objects_y_n)): 
            index_y_n = Y.index(band_objects_y_n[o])
            band_major2_n.append((size.iloc[index_y_n])**weight)
        AWden_n = sum(band_major2_n)
        densities_n.append(AWden_n)
    
    #total density
    densities_n.reverse()
    densities = densities_n+ densities_p
    
    #plot
    ax.plot(np.arange(-band_max, band_max, step),densities, c='black', lw = 3, alpha = 0.6)
    ax.scatter(np.arange(-band_max, band_max, step),densities, c='black', lw=3,  alpha = 0.6,label = 'Strain Density')
    ax.set_xlabel(r'Distance from SBc ($\mu$m)')
    ax.set_ylabel(str(title))
    ax.grid()
    
    ### HALF-WIDTH 
    max_d = np.max(densities)
    
    ###NEGATIVE DENSITIES
    y_n = densities_n
    x_n = np.arange(0, band_max,step) # NB: this function will return positive x values, so the output will have to be converted as OUT-20
    f_n = interp1d( y_n, x_n )
    d_n = f_n(max_d/2)-band_max
    ax.axvline(d_n, c='red', lw=3, label = 'Half-Width', alpha=0.6, zorder=0)
    
    #POSITIVE DENSITIES
    y_p = densities_p
    x_p = np.arange(0, band_max,step)
    f_p = interp1d( y_p, x_p )
    d_p = f_p(max_d/2) - 0
    ax.axvline(d_p, c='red', lw=3, alpha=0.6, zorder=0)
    
    #HALF-WIDTH PRINT
    HW = abs(d_n)+d_p

### 2. Import strain data


In [ ]:
data_set = GD.Open('./Cumulative_shear_strain_cropped_to_2D_CT_scans_for_Claudia.tif')  
band_1 = data_set.GetRasterBand(1) 
b1 = band_1.ReadAsArray()  

X=[]
Y=[]
strain=[]
for (r,c) in [(r,c) for r in range(32) for c in range(20)]: 
    size = b1[r][c]
    X.append(c)
    Y.append(r)
    strain.append(size) 
strain =np.nan_to_num(strain, nan=0.0)

### 3. Thickness of Shear Band from Half-Width

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,5))

density_calculation(int(5*7.91),1, int(-770*7.91), int(200*7.91), y_t.tolist(), pd.DataFrame(strain), r'$\epsilon_D$')

##################### PLOT ##########################
colors = ['black', 'red', 'dodgerblue']
texts = [r'$\epsilon_D$', r'$\omega_{1/2}$', r'SBc']
LS = ['-', '-', '-']
patches = [plt.plot([], [], ls = LS[i], color = colors[i], 
                   label="{:s}".format(texts[i]))[0] for i in range(len(texts))]
ax.legend(handles= patches, labelspacing=0.5, ncol = 3, 
               loc="upper left", bbox_to_anchor = (0.01, 1.17))
ax.annotate(text=r'510 $\mu$m', xy=(-230,0.50))
ax.axvline(x=0, c='dodgerblue', lw = 5, alpha =0.6, zorder=0)
plt.show()

### 4. Rotation of Strain Data and Visualisation of Shear Band Thickness


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,5))
#### DATA
x = np.array(X)
y = np.array(Y)
x_t = []
y_t = []

#ROTATION MATRIX
theta = np.deg2rad(360-60.3)
#theta = np.deg2rad(0)
rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

#TRANSFORM ALL THE DATA SET
for i in range(len(x)): 
    v = np.array([x[i], -y[i]])
    v_t = np.dot(rot, v)
    x_t.append(v_t[0])
    y_t.append(v_t[1])
x_t = np.multiply(41*7.91, x_t)
y_t = np.multiply(41*7.91, y_t)
ax.scatter(x_t, y_t , s=(strain*100)**2, c='black', alpha=0.8)

#PLOT THE LINE

x = np.linspace(-1150*7.91,400*7.91,10)
y = (np.tan(0)*x-770)*7.91
scatter = ax.plot(x, y, c='dodgerblue', lw=5, alpha =0.8)
ax.plot(x, y+510, c='red', lw=3, alpha=0.6)
ax.plot(x, y-510, c='red', lw=3, alpha=0.6)

##### Legend 
## left plot
msizes = [0.01, 0.05, 0.1, 0.35]
markers = []
for size in msizes:
    markers.append(plt.scatter([],[], s=(size*100)**2, label= str(size) , 
                               c='black', alpha=0.8) )
    
ax.legend(handles=markers, columnspacing = 1, handletextpad =0.1, borderpad = 0.35, loc="upper left", ncol = 4,
               bbox_to_anchor = (-0.02, 1.27), title = r'$\epsilon$')

#### Customise Plot 
ax.set_xlabel(r'$\mu$m')
ax.set_ylabel(r'$\mu$m')
ax.grid()
ax.ticklabel_format(style='sci', axis = 'both', scilimits = (0,0))
ax.ticklabel_format(style='sci', axis = 'x', scilimits = (0,0))
 
